# Charlottesville assessment growth by property value

Where are property values rising fastest? Let's check data from the [Charlottesville Open Data Portal](https://opendata.charlottesville.org/) to see how change in assessment relates to property values. First, we'll calculate proportion change in assessments and bin by percentile of 2020 property value.

Note that we're only including single-family attached and single-family detached properties, since the value of a multi-family property is hard to compare to the value of a single-family home. We also exclude properties built before 2010 to simplify proportion change calculations.

In [1]:
%load_ext google.cloud.bigquery

In [2]:
import altair as alt

In [3]:
%%bigquery assessments
with assessments as (
    select
        ParcelNumb,
        TaxYear,
        TotalValue,
        safe_divide(
            (TotalValue - lag(TotalValue, 1) over (partition by ParcelNumb order by TaxYear)),
            lag(TotalValue, 1) over (partition by ParcelNumb order by TaxYear)
        ) as TotalValuePropChange1y,
        safe_divide(
            (TotalValue - lag(TotalValue, 5) over (partition by ParcelNumb order by TaxYear)),
            lag(TotalValue, 5) over (partition by ParcelNumb order by TaxYear)
        ) as TotalValuePropChange5y,
        safe_divide(
            (TotalValue - lag(TotalValue, 10) over (partition by ParcelNumb order by TaxYear)),
            lag(TotalValue, 10) over (partition by ParcelNumb order by TaxYear)
        ) as TotalValuePropChange10y,
    from `cvilledata.cville_open_data.real_estate_all_assessments`
), baseline as (
    select
        ParcelNumb,
        TotalValue,
        ntile(10) over (order by TotalValue) as TotalValueQuintile,
    from `cvilledata.cville_open_data.real_estate_all_assessments`
    where TaxYear = 2020
)
select distinct
    baseline.TotalValueQuintile,
    percentile_cont(assessments.TotalValuePropChange1y, 0.50) over (partition by baseline.TotalValueQuintile) as TotalValuePropChange1y,
    percentile_cont(assessments.TotalValuePropChange5y, 0.50) over (partition by baseline.TotalValueQuintile) as TotalValuePropChange5y,
    percentile_cont(assessments.TotalValuePropChange10y, 0.50) over (partition by baseline.TotalValueQuintile) as TotalValuePropChange10y,
    min(baseline.TotalValue) over (partition by baseline.TotalValueQuintile) as MinValue,
    max(baseline.TotalValue) over (partition by baseline.TotalValueQuintile) as MaxValue,
    concat(
        "Q",
        baseline.TotalValueQuintile,
        ": $",
        min(baseline.TotalValue / 1000) over (partition by baseline.TotalValueQuintile),
        "k",
        "-",
        max(baseline.TotalValue / 1000) over (partition by baseline.TotalValueQuintile),
        "k"
    ) as QuantileLabel,
from assessments
join baseline using (ParcelNumb)
join `cvilledata.cville_open_data.parcel_area_details` details using (ParcelNumb)
join `cvilledata.cville_open_data.real_estate_residential_details` residential using (ParcelNumb)
where
    assessments.TaxYear = 2022
    and residential.UseCode in ('Single Family', 'Single Family Attached')
    and residential.YearBuilt < 2010
order by TotalValueQuintile

Downloading: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:01<00:00,  9.80rows/s]


Now let's look at the results:

In [4]:
assessments

,TotalValueQuintile,TotalValuePropChange1y,TotalValuePropChange5y,TotalValuePropChange10y,MinValue,MaxValue,QuantileLabel
0,1,0.245552,0.441588,0.472244,52700,141600,Q1: $52.7k-141.6k
1,2,0.126412,0.454631,0.510890,141600,199500,Q2: $141.6k-199.5k
2,3,0.132854,0.411304,0.491940,199600,242600,Q3: $199.6k-242.6k
3,4,0.134498,0.407860,0.486631,242600,278100,Q4: $242.6k-278.1k
4,5,0.138117,0.408024,0.509060,278200,311100,Q5: $278.2k-311.1k
5,6,0.123560,0.397079,0.496464,311200,356700,Q6: $311.2k-356.7k
6,7,0.119630,0.398002,0.504797,356900,417800,Q7: $356.9k-417.8k
7,8,0.101566,0.382572,0.492556,417900,525600,Q8: $417.9k-525.6k
8,9,0.089008,0.323116,0.473580,525800,818600,Q9: $525.8k-818.6k
9,10,0.062405,0.297132,0.521957,819100,7320100,Q10: $819.1k-7320.1k


Let's plot one-year proportion change by property value quantile. This shows that cheaper properties increased in value much more than moderate- and high-priced properties:

In [5]:
alt.Chart(assessments).mark_bar().encode(
    x=alt.Y("TotalValuePropChange1y:Q"),
    y=alt.X("QuantileLabel:N", sort=alt.SortField("TotalValueQuintile")),
)

alt.Chart(...)

Does this pattern hold for longer intervals? The distribution of five-year proportion change is somewhat different:

In [6]:
alt.Chart(assessments).mark_bar().encode(
    x=alt.Y("TotalValuePropChange5y:Q"),
    y=alt.X("QuantileLabel:N", sort=alt.SortField("TotalValueQuintile")),
)

alt.Chart(...)

And the pattern is noticeably different for ten-year proportion change, with slightly higher growth in the values of the highest-priced properties.

In [7]:
alt.Chart(assessments).mark_bar().encode(
    x=alt.Y("TotalValuePropChange10y:Q"),
    y=alt.X("QuantileLabel:N", sort=alt.SortField("TotalValueQuintile")),
)

alt.Chart(...)

What does this all tell us?

- One-year assessment changes can be volatile, potentially driven by a small number of sales in a neighborhood. Looking at longer intervals may give more sensible results.
- For longtime property owners, assessments have been growing at a similar pace regardless of property value.
- New owners of lower-priced homes may be experiencing some sticker shock in 2022.